In [57]:
import pandas as pd

df = pd.read_csv("../data/2_base_2800_noise_detected.csv")
len(df)

2800

# 전처리

## 1. 의미 없는 문자 제거

In [58]:
# text 앞뒤에 그래픽/게시판/종합 등의 의미 없는 단어가 나오는 경우 삭제
df["text"] = df["text"].replace(r"^(그래픽|게시판)", "", regex=True)
df["text"] = df["text"].replace(r"^(종합|1보|2보)", "", regex=True)
df["text"] = df["text"].replace(r"(종합|종합1보|종합2보|1보|2보)$", "", regex=True)
df.head()

,ID,text,target,is_noise
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U,4,1
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,1
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야",2,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,1


In [59]:
# 말줄임표: 띄어쓰기 처리 혹은 제거
df["text"] = df["text"].str.replace("…", " ")
df["text"] = df["text"].str.replace("...", "")
df.head()

,ID,text,target,is_noise
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U,4,1
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,1
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야",2,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,5,0
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,1


## 2. 노이즈 문장에서 특수문자 제거

In [60]:
import re
# 영어, 숫자, 한글, 한자 제외 제거
def remove_special_characters(text):
    return re.sub(r"[^a-zA-Z0-9\sㄱ-ㅎㅏ-ㅣ가-힣\u4e00-\u9fff]", "", text)

In [61]:
# 노이즈 데이터에만 함수 적용
df.loc[df["is_noise"] == 1, "text"] = df.loc[df["is_noise"] == 1, "text"].apply(
    remove_special_characters
)

df.head()

,ID,text,target,is_noise
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,5,0
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발 비해 감시 강화,6,1


## 3. 한자는 한글 뜻을 함께 기입하여 사용

In [62]:
from collections import Counter


# 한자 추출 함수 정의
def extract_hanza(text):
    # CJK 통합 한자 범위(U+4E00 ~ U+9FFF)에 해당하는 문자 추출
    return "".join(re.findall("[\u4e00-\u9fff]", text))


# 문자 빈도 축정
def get_char_frequency(text_list):
    # 모든 문자를 하나의 리스트로 합치기
    chars = [char for text in text_list for char in text]
    # Counter를 사용하여 빈도 계산
    char_counts = Counter(chars)
    # 빈도순으로 정렬
    return pd.Series(char_counts).sort_values(ascending=False)

# 한자 빈도수 측정
hanza = df["text"].apply(extract_hanza)

char_frequency = get_char_frequency(hanza)
hanza = list(char_frequency.keys())
print(len(hanza), hanza)

49 ['美', '北', '中', '朴', '靑', '日', '與', '文', '英', '野', '佛', '伊', '獨', '反', '前', '軍', '硏', '對', '外', '社', '黃', '亞', '韓', '株', '車', '崔', '院', '金', '丁', '小', '和', '企', '安', '展', '檢', '親', '銀', '證', '先', '父', '南', '詩', '家', '大', '印', '阿', '故', '州', '重']


In [63]:
hanza_hangul_meaning = [
    ('美', '미국', '나라'), ('北', '북한', '나라'), ('中', '중국', '나라'),
    ('朴', '박', '성씨'), ('靑', '청와대', '정치'), ('日', '일본', '나라'),
    ('與', '여당', '정치'), ('文', '문','성씨'), ('英', '영국', '나라'),
    ('野', '야당', '정치'), ('佛', '프랑스', '나라'), ('伊', '이탈리아', '나라'),
    ('獨', '독일', '나라'), ('反', '반', '일반'), ('前', '전', '일반'),
    ('軍', '군대', '일반'), ('硏', '연구원', '일반'), ('對', '대', '일반'),
    ('外', '외', '일반'), ('社', '회사', '경제'), ('黃', '황','성씨'),
    ('亞', '아시아', '나라'), ('韓', '한국', '나라'), ('株', '주식', '경제'),
    ('車', '차', '일반'), ('崔', '최','성씨'), ('院', '위원', '일반'),
    ('金', '김','성씨'), ('丁', '정','성씨'), ('小', '소', '일반'),
    ('和', '네덜란드', '나라'), ('企', '기업', '경제'), ('安', '안','성씨'),
    ('展', '전시', '일반'), ('檢', '검찰', '정치'), ('親', '친', '일반'),
    ('銀', '은행', '경제'), ('證', '증권', '경제'), ('先', '선', '일반'),
    ('父', '아버지', '일반'), ('南', '남한', '나라'), ('詩', '시', '일반'),
    ('家', '집안', '일반'), ('大', '대', '일반'), ('印', '인도', '나라'),
    ('阿', '아프리카', '나라'), ('故', '고', '일반'), ('州', '주', '나라'),
    ('重', '중공업', '경제')
]

def annotate_hanzi_with_meaning(text):
    for hanzi, hangul, _ in hanza_hangul_meaning:
        text = text.replace(hanzi, f"{hanzi}({hangul})")
    return text

In [64]:
df.loc[:, "text"] = df["text"].apply(annotate_hanzi_with_meaning)
df.head(10)

,ID,text,target,is_noise
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,5,0
4,ynat-v1_train_00004,pI美(미국)대선I앞두고 R2fr단 발 비해 감시 강화,6,1
5,ynat-v1_train_00005,美(미국)성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,0
6,ynat-v1_train_00006,프로야구롯TKIAs광주 경기 y천취소,1,1
7,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파 3위 굳,4,0
8,ynat-v1_train_00008,朴(박)대통령 얼마나 많이 놀라셨어요 경주 지진현장 방문,6,0
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔 알뜰폰 기대감,4,0


In [25]:
df.to_csv("3_d_2800_hanzi_dictionary.csv", index=False)

## 4. 형태소 분석기로 노이즈 제거

In [95]:
df = pd.read_csv("../data/3_d_2800_hanzi_dictionary.csv")

In [96]:
df = df[df["is_noise"]==1]
print(len(df))
df.head()

1606


,ID,text,target,is_noise
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1
4,ynat-v1_train_00004,pI美(미국)대선I앞두고 R2fr단 발 비해 감시 강화,6,1
6,ynat-v1_train_00006,프로야구롯TKIAs광주 경기 y천취소,1,1


In [97]:
# 형태소 분석기 임포트
from konlpy.tag import *
okt = Okt()

In [98]:
df["pos_tagging"] = df["text"].copy()
# 단어별 구분 없이 한 문장 통으로 하려면
# df["pos_tagging"] = df["pos_tagging"].apply(lambda text: okt.pos(text))
# 단어별로 형태소 분석 적용
df["pos_tagging"] = df["pos_tagging"].apply(str.split)
df["pos_tagging"] = df["pos_tagging"].apply(lambda li: [okt.pos(e) for e in li])

In [99]:
df.head()

,ID,text,target,is_noise,pos_tagging
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1,"[[(정, Noun), (i, Alpha)], [(파, Noun), (1, Numb..."
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1,"[[(K, Alpha), (찰국, Noun), (DLwo, Alpha)], [(로,..."
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1,"[[(m, Alpha)], [(김정, Noun)], [(자주, Noun), (통일,..."
4,ynat-v1_train_00004,pI美(미국)대선I앞두고 R2fr단 발 비해 감시 강화,6,1,"[[(pI, Alpha), (美, Foreign), ((, Punctuation),..."
6,ynat-v1_train_00006,프로야구롯TKIAs광주 경기 y천취소,1,1,"[[(프로야구, Noun), (롯, Noun), (TKIAs, Alpha), (광주..."


In [100]:
def join_words(morphs_lists):
    def filter_and_join_morphemes(morphological_results):
        # Alpha와 Punctuation을 제외한 문자만 필터링
        filtered_characters = [
            word for word, pos in morphological_results if pos not in ["Alpha", "Number"]
        ]
        # 필터링된 문자들을 공백으로 구분하여 문자열로 결합
        return "".join(filtered_characters)
    return " ".join(
        [filter_and_join_morphemes(morphs_list) for morphs_list in morphs_lists]
    )


df["pos_processed"] = df["pos_tagging"].apply(join_words)
df.head()

,ID,text,target,is_noise,pos_tagging,pos_processed
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1,"[[(정, Noun), (i, Alpha)], [(파, Noun), (1, Numb...",정 파 미사 이용기간 단 분종
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1,"[[(K, Alpha), (찰국, Noun), (DLwo, Alpha)], [(로,...",찰국 로한 회장 송
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1,"[[(m, Alpha)], [(김정, Noun)], [(자주, Noun), (통일,...",김정 자주통일 새열나가야
4,ynat-v1_train_00004,pI美(미국)대선I앞두고 R2fr단 발 비해 감시 강화,6,1,"[[(pI, Alpha), (美, Foreign), ((, Punctuation),...",美(미국)대선앞두고 단 발 비해 감시 강화
6,ynat-v1_train_00006,프로야구롯TKIAs광주 경기 y천취소,1,1,"[[(프로야구, Noun), (롯, Noun), (TKIAs, Alpha), (광주...",프로야구롯광주 경기 천취소


In [101]:
df["pos_processed"] = df["pos_processed"].str.strip()
df["pos_processed"] = df["pos_processed"].apply(lambda x: re.sub(r"\s+", " ", x))

In [102]:
df = df[["ID", "pos_processed", "target","is_noise"]].rename(columns={"pos_processed": "text"})
df.to_csv("4_d_1606_only_noise.csv", index=False)